### Database creation and loading

##### Initiate driver and connect to local database

In [1]:
import neo4j
from neo4j import GraphDatabase

# Establishing connection with remote db
URI = "bolt://localhost:7687"
AUTH = ("dbms_user", "dbms1234")

driver = GraphDatabase.driver(URI, auth=AUTH)
driver.verify_connectivity()

##### Create database named flights

In [2]:
query = """
CREATE OR REPLACE DATABASE flights
"""

driver.execute_query(query)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x0000028F955EE650>, keys=[])

##### To make ingestion faster, create index on airport code for airport nodes (two or modes nodes cannot have same aiport code)

In [3]:
query = """
CREATE INDEX airport_code FOR (n:airport) ON (n.airportID)
"""

driver.execute_query(query,database_='flights')

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x0000028F955EDF90>, keys=[])

##### Load CSV file (flights_neo4j.csv) into database flights

In [4]:
query = """
LOAD CSV WITH HEADERS FROM 'file:///flights_neo4j.csv' AS row  FIELDTERMINATOR ','
MERGE (a:airport {airportID: row.Dest})
MERGE (b:airport {airportID: row.Origin})
WITH a,b,row
CREATE (a)-[f:CONNECTS_TO]->(b)
SET f.CRSDepTime = toInteger(row.CRSDepTime), 
f.Marketing_Airline_Network = row.Marketing_Airline_Network,
f.CRSElapsedTime = toInteger(row.CRSElapsedTime)
"""

driver.execute_query(query,database_='flights')

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x0000028F955FB3D0>, keys=[])

### Data manipulation

##### Find the top 5 airports with the highest number of departure flights.

In [5]:
query = """
MATCH (a:airport)-[r:CONNECTS_TO]->() 
RETURN a.airportID as origin_airport, count(r) as n_of_flights
ORDER BY  n_of_flights desc
LIMIT 5
"""

driver.execute_query(query, result_transformer_=neo4j.Result.to_df, database_='flights')

,origin_airport,n_of_flights
0,ATL,936
1,DEN,881
2,ORD,848
3,DFW,835
4,CLT,585


##### Find the top 5 airports with the highest number of American Airlines departure flights. 

In [6]:
query = """
MATCH (a:airport)-[r:CONNECTS_TO {Marketing_Airline_Network:'AA'}]->(b:airport) 
RETURN a.airportID as origin_airport, count(r) as n_of_flights
ORDER BY  n_of_flights desc
LIMIT 5
"""

driver.execute_query(query, result_transformer_=neo4j.Result.to_df, database_='flights')

,origin_airport,n_of_flights
0,DFW,718
1,CLT,502
2,ORD,311
3,DCA,252
4,PHL,216


##### Find the top 5 airports with the highest number of flights going to JFK.

In [7]:
query = """
MATCH p=(a:airport)-[r:CONNECTS_TO]->(b:airport {airportID:"JFK"}) 
RETURN a.airportID as origin_airport, count(p) as n_of_flights
ORDER BY  n_of_flights desc
LIMIT 5
"""

driver.execute_query(query, result_transformer_=neo4j.Result.to_df, database_='flights')

,origin_airport,n_of_flights
0,LAX,30
1,SFO,22
2,BOS,16
3,MIA,15
4,MCO,14


##### Find all the airports that have direct flight from Trenton airport (TTN). Sort results by destination airport code.

In [8]:
query = """
MATCH (a:airport {airportID:"TTN"})-[:CONNECTS_TO]->(b) 
RETURN DISTINCT b.airportID as destination_airport
ORDER BY b.airportID 
"""

driver.execute_query(query, result_transformer_=neo4j.Result.to_df, database_='flights')

,destination_airport
0,ATL
1,CLT
2,MCO
3,TPA


##### Find all the airports that have direct flight from Trenton airport (TTN). Sort results by flight duration in descending order.

In [9]:
query = """
MATCH (a:airport {airportID:"TTN"})-[r:CONNECTS_TO]->(b) 
RETURN DISTINCT b.airportID as destination_airport, r.CRSElapsedTime as flight_time
ORDER BY r.CRSElapsedTime DESC
"""

driver.execute_query(query, result_transformer_=neo4j.Result.to_df, database_='flights')

,destination_airport,flight_time
0,TPA,168
1,MCO,153
2,MCO,152
3,ATL,132
4,CLT,106


##### Find all the airports that have flights from Trenton airport (TTN) with one stop. Sort results by airport code.

In [10]:
query = """
MATCH (a:airport {airportID:"TTN"})-[:CONNECTS_TO*2]->(b) 
RETURN DISTINCT b.airportID as destination_airport
ORDER BY b.airportID 
"""

driver.execute_query(query, result_transformer_=neo4j.Result.to_df, database_='flights')

,destination_airport
0,ABE
1,ABQ
2,ABY
3,ACY
4,AEX
...,...
155,TUS
156,TYS
157,VLD
158,VPS


##### Find all the airports that have flights from Trenton airport (TTN) with one stop. Sort results by minimum time in the air (sum of flights duration) in desc order.

In [11]:
query = """
MATCH (a:airport {airportID:"TTN"})-[r:CONNECTS_TO*2]->(b) 
WITH b.airportID as destination_airport, r[0].CRSElapsedTime + r[1].CRSElapsedTime as time_in_air
RETURN destination_airport, MIN(time_in_air) as min_time_in_air
ORDER BY min_time_in_air 
"""

driver.execute_query(query, result_transformer_=neo4j.Result.to_df, database_='flights')

,destination_airport,min_time_in_air
0,GSO,159
1,AVL,164
2,GSP,164
3,CAE,166
4,CHS,168
...,...,...
155,SEA,408
156,SFO,409
157,SJC,419
158,ANC,548


### Run Dijkstra Shortest Path Algorithm

##### Create a graph projection

In [12]:
query = """

CALL gds.graph.project(
    'myGraph',
    'airport',
    'CONNECTS_TO',
    {
        relationshipProperties: 'CRSElapsedTime'
    }
)


"""

driver.execute_query(query, result_transformer_=neo4j.Result.to_df, database_='flights')

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'airport': {'label': 'airport', 'properties':...","{'CONNECTS_TO': {'aggregation': 'DEFAULT', 'or...",myGraph,330,20019,37


##### Run algorithm to find the shortest path between Trenton (New Jersey) and Anchorage (Alaska) airports, weighted on flight time duration

In [13]:
query = """

MATCH (start:airport{airportID:"TTN"}), (end:airport{airportID:"ANC"})
CALL gds.shortestPath.dijkstra.stream('myGraph', {
    sourceNode: start,
    targetNode: end,
    relationshipWeightProperty: 'CRSElapsedTime'
})
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    index,
    gds.util.asNode(sourceNode).airportID AS sourceNodeName,
    gds.util.asNode(targetNode).airportID AS targetNodeName,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).airportID] AS nodeNames,
    costs,
    nodes(path) as path
ORDER BY index


"""

driver.execute_query(query,  result_transformer_=neo4j.Result.to_df, database_='flights')


,index,sourceNodeName,targetNodeName,totalCost,nodeNames,costs,path
0,0,TTN,ANC,548.0,"[TTN, ATL, ANC]","[0.0, 132.0, 548.0]","[(airportID), (airportID), (airportID)]"
